Скачайте корпус текстов, обработайте его с помощью UDPipe, извлеките все группы 
"глагол + прямое дополнение, выраженное существительным" 
(не учитывайте глаголы, которые встречаются в корпусе менее 50 раз).


In [2]:
!C:/Users/User/Documents/nlp/udpipe-1.2.0-bin/bin-win64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:/Users/User/Documents/nlp/russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

Loading UDPipe model: done.


In [3]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [4]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [5]:
import re

with open ('testset2.conllu', encoding='utf-8') as f:
    text = [line for line in f.readlines() if re.search(r'^[0-9]', line)]
    tokens = [line.split("\t") for line in text if 'PUNCT' not in line]
    tokens = [(line[2], line[3]) for line in tokens]

In [6]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    return lemmas

In [7]:
def get_ngrams(tokens, n=2, patterns=None):
    """
    Если patterns не None, давайте проверять, что части речи биграммы есть в patterns.
    Например, patterns = (['ADJF', 'NOUN'])
    Подумаем о том, хотим ли мы склеивать два слова из разных предложений.
    Можно использовать itertools.islice
    """
    ngrams = []
    for i in range(len(tokens) - (n - 1)):
        ngram = [tok[0] for tok in tokens[i:i+n]]
        tags = [tok[1] for tok in tokens[i:i+n]]
        if patterns is not None:
            if tags in patterns:
                ngrams.append(ngram)
        else:
            ngrams.append(ngram)
    return ngrams

In [16]:
bigrams = get_ngrams(tokens, patterns=[['VERB', 'NOUN']])
bigrams

[['запускать', 'механизм'],
 ['рассматривать', 'дело'],
 ['рассмотреть', 'вопрос'],
 ['определять', 'координата'],
 ['терпеть', 'бедствие'],
 ['начаться', 'слушание'],
 ['определить', 'срок'],
 ['вызвать', 'недовольство'],
 ['привести', 'обитатель'],
 ['получить', 'срок'],
 ['собирать', 'доказательство'],
 ['обвинить', 'олигарх'],
 ['предать', 'суд'],
 ['передать', 'судья'],
 ['возложить', 'правительство'],
 ['идти', 'речь'],
 ['просить', 'суд'],
 ['рассматривать', 'ходатайство'],
 ['обжаловать', 'постановление'],
 ['причинять', 'нарушение'],
 ['затруднять', 'доступ'],
 ['отозвать', 'иск'],
 ['отозвать', 'иск'],
 ['предоставлять', 'норвежец'],
 ['сообщить', 'источник'],
 ['иметь', 'вид'],
 ['соответствовать', 'действительность'],
 ['сообщить', 'адвокат'],
 ['избрать', 'мера'],
 ['быть', 'основание'],
 ['оказать', 'давление'],
 ['воспрепятствовать', 'расследование'],
 ['дать', 'согласие'],
 ['возбуждать', 'дело'],
 ['признать', 'сделка'],
 ['обвинить', 'господин'],
 ['удовлетворить', 'ж

Оцените полученные словосочетания следующими метриками: log-likelihood, dice, PMI (можно использовать nltk.collocations).

In [15]:
bigrams

<function nltk.util.bigrams(sequence, **kwargs)>

In [9]:
from collections import Counter

In [10]:
import nltk

In [11]:
from nltk.collocations import *

In [12]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = nltk.collocations.BigramCollocationFinder.from_words([tok[0] for tok in tokens])

In [17]:
scores_dice = [pair for pair in finder2.score_ngrams(bigram_measures.dice) if list(pair[0]) in bigrams][:100]
scores_pmi = [pair for pair in finder2.score_ngrams(bigram_measures.pmi) if list(pair[0]) in bigrams][:100]
scores_like = [pair for pair in finder2.score_ngrams(bigram_measures.likelihood_ratio) if list(pair[0]) in bigrams][:100]

In [18]:
scores_dice

[(('Нитрохй', 'дистрибьюшн'), 1.0),
 (('Осуждеть', 'чтить'), 1.0),
 (('Санди', 'мейл'), 1.0),
 (('ассоциировать', 'Тнк'), 1.0),
 (('взорванный', 'террористкойсмертница'), 1.0),
 (('воплощать', 'бытие'), 1.0),
 (('выкрадеть', 'видеопленка'), 1.0),
 (('должать', 'попыток'), 1.0),
 (('завязать', 'потасовка'), 1.0),
 (('заделывать', 'ямища'), 1.0),
 (('заняться', 'контрагитация'), 1.0),
 (('запускать', 'механизм'), 1.0),
 (('заразиться', 'туберкулез'), 1.0),
 (('заставлять', 'дорожник'), 1.0),
 (('наслаждаться', 'красота'), 1.0),
 (('обеспокоить', 'благосостояние'), 1.0),
 (('облагородить', 'капитализм'), 1.0),
 (('оплачеть', 'квитанция'), 1.0),
 (('ослепить', 'фара'), 1.0),
 (('оставлять', 'вакуум'), 1.0),
 (('охладить', 'пыл'), 1.0),
 (('перегибать', 'палка'), 1.0),
 (('поедать', 'шпинат'), 1.0),
 (('прекращеть', 'Обвинитель'), 1.0),
 (('преуменьшить', 'зверство'), 1.0),
 (('пробить', 'ограждение'), 1.0),
 (('просроченный', 'транзит'), 1.0),
 (('развеваться', 'полотнище'), 1.0),
 (('разр

Подготовьте "золотой стандарт" коллокаций (далее ЗС) для этого корпуса: возьмите словосочетания, 
    которые попадают в топ-100 по всем метрикам, пересеките со словарем глагольной сочетаемости.

In [19]:
top = list(set(scores_dice + scores_pmi + scores_like))

In [20]:
!C:/Users/User/Documents/nlp/udpipe-1.2.0-bin/bin-win64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:/Users/User/Documents/nlp/russian-syntagrus-ud-2.4-190531.udpipe \
< verb_coll.txt > verb_lemm.conllu

Loading UDPipe model: done.


In [21]:
with open ('verb_lemm.conllu', encoding='utf-8') as f:
    word_pairs = []
    text = [l.split("SpacesAfter=\\t\n")[-1] for l in f.read().split("SpacesAfter=\\n\n") if l]
    for line in text:
        inlines = [l.split("\t") for l in line.split("\n") if l]
        inlines = [l for l in inlines if re.search(r'^[0-9]', l[0])]
        if len(inlines) == 2:
            word_pairs.append((inlines[0][2], inlines[1][2]))

In [22]:
word_pairs

[('аборт', 'делать'),
 ('делать', 'аборт'),
 ('делать', 'аборт'),
 ('авария', 'произойти'),
 ('авария', 'случиться'),
 ('потерпеть', 'авария'),
 ('произойти', 'авария'),
 ('случиться', 'авария'),
 ('завоевать', 'авторитет'),
 ('подрыть', 'авторитет'),
 ('подрыть', 'авторитет'),
 ('пользоваться', 'авторитет'),
 ('оспаривать', 'акт'),
 ('активность', 'проявлять'),
 ('активность', 'проявлять'),
 ('проявить', 'активность'),
 ('проявить', 'активность'),
 ('проявлять', 'активность'),
 ('проявлять', 'активность'),
 ('акцент', 'делать'),
 ('акцент', 'переносить'),
 ('акцент', 'делать'),
 ('делать', 'акцент'),
 ('расставлять', 'акцент'),
 ('расставлять', 'акцент'),
 ('расставлять', 'акцент'),
 ('делать', 'акцент'),
 ('сместить', 'акцент'),
 ('сместить', 'акцент'),
 ('предлагать', 'алгоритм'),
 ('предлагать', 'алгоритм'),
 ('разработать', 'алгоритм'),
 ('разработать', 'алгоритм'),
 ('вызывать', 'аллергия'),
 ('вызывать', 'аллергия'),
 ('найти', 'альтернатива'),
 ('найти', 'альтернатива'),
 ('ана

In [23]:
sort_top = [line for line in top if line[0] in word_pairs]

In [24]:
sort_top

[(('оспаривать', 'решение'), 71.97322086523803),
 (('продлить', 'срок'), 100.57269338556176),
 (('хранить', 'молчание'), 15.501598788870902),
 (('представлять', 'интерес'), 298.3111880424806),
 (('соответствовать', 'действительность'), 99.14782446838146),
 (('оказать', 'давление'), 65.70004822301398),
 (('упустить', 'выгода'), 0.5),
 (('прекратить', 'производство'), 46.229445984788676),
 (('идти', 'речь'), 42.91001525240716),
 (('вынести', 'приговор'), 135.92698508708463),
 (('рассмотреть', 'жалоба'), 116.24694538424626),
 (('провести', 'обыск'), 42.80479222616313),
 (('вынести', 'постановление'), 62.855590117521096),
 (('возместить', 'ущерб'), 45.15704959480961),
 (('начаться', 'процесс'), 49.15253791376535),
 (('нанести', 'ущерб'), 87.1592379741507),
 (('наложить', 'арест'), 148.35247463648784),
 (('вести', 'переговоры'), 49.14923760596165),
 (('принимать', 'участие'), 68.40017519007242),
 (('провести', 'проверка'), 55.59719088700345),
 (('опубликовать', 'опровержение'), 91.036709490

Добавьте в ЗС словосочетания из топ-100 , которые не вошли в словарь, но являются коллокациями (если такие есть), 
объясните свой выбор.

In [28]:
nogold = [line for line in top if line not in sort_top]
nogold

[(('подъехать', 'ПАЗик'), 16.08656128959206),
 (('дать', 'киллер'), 41.20270504705551),
 (('комментировать', 'решение'), 66.88386574556708),
 (('наслаждаться', 'красота'), 1.0),
 (('усматривать', 'признак'), 62.54063227818544),
 (('перекрыть', 'полгород'), 16.08656128959206),
 (('аудит', 'юкос'), 125.40015074608314),
 (('заниматься', 'сбыт'), 58.94140124803248),
 (('отреставрировать', 'корпус'), 15.086561289592058),
 (('ослепить', 'фара'), 17.08656128959206),
 (('судетраница', '|МТС'), 1.0),
 (('производить', 'госрегистрация'), 15.501598788870902),
 (('сработать', 'мэтр'), 15.501598788870902),
 (('исследовать', 'ролик'), 15.086561289592058),
 (('перегибать', 'палка'), 17.08656128959206),
 (('огласить', 'приговор'), 50.385920563537155),
 (('развеваться', 'полотнище'), 17.08656128959206),
 (('ассоциировать', 'Тнк'), 1.0),
 (('Осуждеть', 'чтить'), 17.08656128959206),
 (('проливать', 'кровь'), 0.5),
 (('устанавливать', 'презумпция'), 0.5),
 (('отравить', 'зонтика'), 0.5),
 (('преподнести',

In [29]:
sort_top += [
  (('комментировать', 'решение'), 66.88386574556708),
  (('наслаждаться', 'красота'), 1.0),
  (('перегибать', 'палка'), 17.08656128959206),
  (('огласить', 'приговор'), 50.385920563537155),
  (('проливать', 'кровь'), 0.5),
  (('иметь', 'право'), 189.03553504456767),
  (('отклонить', 'иск'), 110.85386875573855),
  (('подать', 'апелляция'), 116.51807466550632),
  (('придавать', 'огласка'), 16.08656128959206),
  (('удовлетворить', 'ходатайство'), 227.81434468255617),
  (('отозвать', 'иск'), 44.89222084531106),
  (('запускать', 'механизм'), 1.0),
  (('сгуститься', 'туча'), 16.08656128959206),
  (('держаться', 'молодец'), 16.08656128959206),
  (('выдать', 'санкция'), 105.61010964756716),
  (('растягивать', 'удовольствие'), 0.5),
  (('надеть', 'наручники'), 0.5),
  (('тянуться', 'шлейф'), 0.5),
  (('уплатить', 'налог'), 55.4641345643287),
  (('заразиться', 'туберкулез'), 1.0),
  (('признать', 'правота'), 71.50369002919983),
  (('расклеить', 'листовка'), 0.5),
  (('оспорить', 'решение'), 44.737426431348744),
  (('наслаждаться', 'красота'), 17.08656128959206) 
]

Оцените ранговую корреляцию (коэффициент Спирмена) результатов по каждой метрике с ЗС. Как это работает, читайте, например, тут. 
Можно использовать scipy.stats.spearmanr. Опишите ошибки каждой метрики.